In [1]:
import os 

In [2]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project/research'

In [3]:
os.chdir('/Users/sunilverma/Text-Summarizer-Project')

In [4]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig: 
    root_dir: Path 
    data_path: Path
    tokenizer_name: str

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [17]:
from pathlib import Path
from text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir]) 
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )
        return data_transformation_config

In [9]:
import os 
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config 
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)


    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self): 
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features, 
            batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [22]:
try: 
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

2025-12-17 22:38:14,645: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-17 22:38:14,665: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-17 22:38:14,667: INFO: common: created directory at: artifacts
2025-12-17 22:38:14,672: INFO: common: created directory at: artifacts/data_transformation


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 130837.08 examples/s]
